## Logistic Regression + Factorization Machines (FM/FFM) for Avazu CTR

---

**Goal:** Train a strong CTR model on the Avazu dataset using:
- Baseline **Logistic Regression** (sparse one-hot / hashing features)
- **Factorization Machines (FM)** with **logistic loss** via `fastFM`


#### Imports & Config

---

...

In [1]:
from pathlib import Path
from typing import Self

import numpy as np
import pandas as pd
from scipy import sparse as sp
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction import FeatureHasher
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    average_precision_score,
    log_loss,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.utils.validation import check_is_fitted, check_random_state

#### Paths & Avazu schema

---

- Download Avazu `"train"` CSV (from Kaggle) and set `AVAZU_PATH` to the file.
- We will use **day-based split** by parsing the `hour` column (format `YYYYMMDDHH`) and selecting the **last day as validation** to avoid temporal leakage.


In [2]:
DATA_DIR = Path("../data/raw/avazu")
TRAIN_FILE = DATA_DIR / "train.gz"
TEST_FILE = DATA_DIR / "test.gz"

RANDOM_STATE = 42
SAMPLE_FRAC = 0.05

DTYPE_COLS = {
    "id": np.int64,
    "click": np.int8,
    "hour": np.int64,
    "C1": np.int32,
    "banner_pos": np.int8,
    "site_id": "category",
    "site_domain": "category",
    "site_category": "category",
    "app_id": "category",
    "app_domain": "category",
    "app_category": "category",
    "device_id": "category",
    "device_ip": "category",
    "device_model": "category",
    "device_type": np.int8,
    "device_conn_type": np.int8,
    "C14": np.int32,
    "C15": np.int32,
    "C16": np.int32,
    "C17": np.int32,
    "C18": np.int32,
    "C19": np.int32,
    "C20": np.int32,
    "C21": np.int32,
}

# Columns to treat as categorical (strings). `hour` will be parsed into day/hour.
CAT_COLS = [
    "banner_pos",
    "site_id",
    "site_domain",
    "site_category",
    "app_id",
    "app_domain",
    "app_category",
    "device_id",
    "device_ip",
    "device_model",
    "device_type",
    "device_conn_type",
    "C14",
    "C15",
    "C16",
    "C18",
    "C19",
    "C20",
    "C21",
]
TARGET_COL = "click"

ID_COL = "id"
TARGET_COL = "click"
DATETIME_COL = "hour"
DATETIME_FORMAT = "%y%m%d%H"

TEST_SIZE = 0.2
HASH_N_FEATURES = 2**22

#### Utilities: day-based split & chunked reader

---

We split **by last day** (validation) to mimic online production and avoid target leakage across time.

In [3]:
def read_csv_infer(path: Path, usecols: list[str] = None) -> pd.DataFrame:
    return pd.read_csv(path, usecols=usecols, dtype=DTYPE_COLS, compression="infer")


df = read_csv_infer(TRAIN_FILE)
print(f"Train read: shape={df.shape}")

df = df.sample(frac=SAMPLE_FRAC, random_state=RANDOM_STATE)
print(f"Sampled df to fraction={SAMPLE_FRAC}: shape={df.shape}")

Train read: shape=(40428967, 24)
Sampled df to fraction=0.05: shape=(2021448, 24)


In [4]:
def add_time_features(df: pd.DataFrame) -> pd.DataFrame:
    s = df[DATETIME_COL].astype(str).str.zfill(8)
    ts = pd.to_datetime(s, format=DATETIME_FORMAT, errors="coerce", utc=True)
    return df.assign(hod=ts.dt.hour, dow=ts.dt.day_of_week)


df = add_time_features(df)

#### Feature engineering for FM and Logistic Regression (hashing)

---

We will use a **hashing trick** to map `field=value` pairs to a fixed-dimensional sparse space.
- This is memory-friendly and scales to Avazu.
- FM will model pairwise interactions internally; Logistic Regression will not (baseline).

> You can tune `N_FEATS` (e.g., `2**24` or `2**22`) depending on your RAM.

In [5]:
COLS_TO_DROP = [
    ID_COL,
    TARGET_COL,
    DATETIME_COL,
    "C1",
    "C17",
]

X = df.drop(columns=COLS_TO_DROP, axis=1)
y = df[TARGET_COL]

In [6]:
def make_cat_dicts(X: pd.DataFrame) -> list[dict[str, int]]:
    return [
        {f"{c}={getattr(row, c)}": 1 for c in list(X.columns)}
        for row in X.itertuples(index=False)
    ]


hashing_pipeline = Pipeline(
    steps=[
        (
            "cat_pairs",
            FunctionTransformer(func=make_cat_dicts, validate=False),
        ),
        (
            "cat_hash",
            FeatureHasher(n_features=HASH_N_FEATURES, alternate_sign=False),
        ),
    ]
)
featurizer = ColumnTransformer(
    transformers=[
        ("cats", hashing_pipeline, CAT_COLS),
    ],
    remainder="drop",
    sparse_threshold=1.0,  # keep sparse
)

#### Baseline: Logistic Regression

---

A strong, well-regularized baseline using sparse hashed features.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
)
print(f"X train: {X_train.shape=}")
print(f"y train: {y_train.shape=}")

X train: X_train.shape=(1617158, 21)
y train: y_train.shape=(1617158,)


In [8]:
# Baseline logistic regression (SAG/SAGA handle sparse well)
# Tip: On very high-dimensional hashing, 'saga' with l1 or elasticnet can be strong.
lr = LogisticRegression(
    penalty="l2",
    # class_weight="balanced",  # negatively affects the log loss
    random_state=RANDOM_STATE,
    solver="saga",
    max_iter=1000,
    verbose=2,
    n_jobs=-1,
)
lr_pipeline = Pipeline(
    steps=[
        ("features", featurizer),
        ("clf", lr),
    ]
)

lr_pipeline.fit(X_train, y_train)
y_pred_proba = lr_pipeline.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba >= 0.5).astype(int)

print("LogLoss (LR):", log_loss(y_test, y_pred_proba))
print("ROC-AUC (LR):", roc_auc_score(y_test, y_pred_proba))
print("PR-AUC  (LR):", average_precision_score(y_test, y_pred_proba))
print("Accuracy  (LR):", accuracy_score(y_test, y_pred))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.


Epoch 1, change: 1
Epoch 2, change: 0.19851891
Epoch 3, change: 0.11275568
Epoch 4, change: 0.075374844
Epoch 5, change: 0.056062811
Epoch 6, change: 0.049849831
Epoch 7, change: 0.049375518
Epoch 8, change: 0.034196703
Epoch 9, change: 0.034028242
Epoch 10, change: 0.030705083
Epoch 11, change: 0.023626997
Epoch 12, change: 0.031298328
Epoch 13, change: 0.016373771
Epoch 14, change: 0.015072256
Epoch 15, change: 0.014034818
Epoch 16, change: 0.012993055
Epoch 17, change: 0.012024488
Epoch 18, change: 0.011132935
Epoch 19, change: 0.0093937501
Epoch 20, change: 0.0090100443
Epoch 21, change: 0.007372683
Epoch 22, change: 0.0069090339
Epoch 23, change: 0.006428534
Epoch 24, change: 0.0058763374
Epoch 25, change: 0.0051660198
Epoch 26, change: 0.0048409828
Epoch 27, change: 0.0044553124
Epoch 28, change: 0.0041379696
Epoch 29, change: 0.0038310991
Epoch 30, change: 0.0034218334
Epoch 31, change: 0.0031736996
Epoch 32, change: 0.0029503283
Epoch 33, change: 0.0026545058
Epoch 34, change: 

#### Factorization Machines (FM) — logistic loss via `fastFM`

---

- We reuse the same hashed one-hot features (no manual cross-features; FM learns them).
- `fastFM.sgd.FMClassification` expects labels in **{-1, +1}** for logit classification.


In [ ]:
class FactorizationMachineClassifier(BaseEstimator, ClassifierMixin):
    def __init__(
        self,
        rank: int = 16,
        n_epochs: int = 10,
        learning_rate: float = 0.05,
        l2_w: float = 1e-6,
        l2_V: float = 1e-6,
        init_stdev: float = 0.1,
        fit_intercept: bool = True,
        shuffle: bool = True,
        random_state: int = RANDOM_STATE,
        verbose: int = 1,
    ) -> None:
        self.rank = rank
        self.n_epochs = n_epochs
        self.learning_rate = learning_rate
        self.l2_w = l2_w
        self.l2_V = l2_V
        self.init_stdev = init_stdev
        self.fit_intercept = fit_intercept
        self.shuffle = shuffle
        self.random_state = random_state
        self.verbose = verbose

    @staticmethod
    def _sigmoid(z: np.ndarray) -> np.ndarray:
        return 1.0 / (1.0 + np.exp(-z))

    def _init_params(self, n_features: int, rng: np.random.RandomState) -> None:
        self.w0_ = 0.0
        self.w_ = np.zeros(n_features, dtype=np.float64)
        self.V_ = rng.normal(0.0, self.init_stdev, size=(n_features, self.rank)).astype(
            np.float64
        )

    def _decision_function_row(self, indices: np.ndarray, data: np.ndarray) -> float:
        eta = self.w0_ if self.fit_intercept else 0.0
        if indices.size == 0:
            return float(eta)

        # Linear term
        eta += np.dot(self.w_[indices], data)

        # Efficient pairwise interactions
        V_slice = self.V_[indices, :]  # (nnz, k)
        p = V_slice.T @ data  # (k,)
        sum_vx_sq = np.sum((V_slice * V_slice) * (data[:, None] ** 2), axis=0)  # (k,)
        eta += 0.5 * (np.sum(p * p) - np.sum(sum_vx_sq))
        return float(eta)

    def decision_function(self, X: sp.csr_matrix) -> np.ndarray:
        check_is_fitted(self, attributes=["w_", "V_"])

        if not sp.isspmatrix_csr(X):
            X = sp.csr_matrix(X)

        n_samples = X.shape[0]
        out = np.empty(n_samples, dtype=np.float64)
        for r in range(n_samples):
            start, end = X.indptr[r], X.indptr[r + 1]
            out[r] = self._decision_function_row(
                X.indices[start:end], X.data[start:end]
            )
        return out

    def predict_proba(self, X: sp.csr_matrix) -> np.ndarray:
        eta = self.decision_function(X)
        p1 = self._sigmoid(eta)
        return np.vstack([1.0 - p1, p1]).T

    def predict(self, X: sp.csr_matrix) -> np.ndarray:
        return (self.predict_proba(X)[:, 1] >= 0.5).astype(np.int32)

    def fit(self, X: sp.csr_matrix, y: np.ndarray) -> Self:
        if not sp.isspmatrix_csr(X):
            X = sp.csr_matrix(X)

        y = np.asarray(y, dtype=np.int8)
        if set(np.unique(y)) - {0, 1}:
            raise ValueError("y must be binary {0,1}.")

        n_samples, n_features = X.shape
        rng = check_random_state(self.random_state)
        self._init_params(n_features, rng)

        order = np.arange(n_samples)
        lr = float(self.learning_rate)

        for epoch in range(self.n_epochs):
            if self.shuffle:
                rng.shuffle(order)

            total_loss = 0.0

            for r in order:
                start, end = X.indptr[r], X.indptr[r + 1]
                idx = X.indices[start:end]
                dat = X.data[start:end]

                # Forward (handles empty row)
                eta = self._decision_function_row(idx, dat)
                p = self._sigmoid(np.array([eta]))[0]
                e = p - y[r]  # gradient w.r.t. eta
                total_loss += -(
                    y[r] * np.log(p + 1e-15) + (1 - y[r]) * np.log(1 - p + 1e-15)
                )

                if self.fit_intercept:
                    self.w0_ -= lr * e

                if idx.size == 0:
                    continue

                # w-gradient: e * x_i + l2_w * w_i
                self.w_[idx] -= lr * (e * dat + self.l2_w * self.w_[idx])

                # V-gradient (vectorized over active indices)
                V_slice = self.V_[idx, :]  # (nnz, k)
                p_k = V_slice.T @ dat  # (k,)
                # grad_V_slice = e * [ x_i * (p_k - v_i * x_i) ] + l2_V * v_i
                grad_V_slice = (
                    e * (dat[:, None] * (p_k[None, :] - V_slice * dat[:, None]))
                    + self.l2_V * V_slice
                )
                V_slice -= lr * grad_V_slice
                self.V_[idx, :] = V_slice

            if self.verbose:
                print(
                    f"[FM] Epoch {epoch + 1}/{self.n_epochs}, LogLoss: {total_loss / n_samples:.6f}"
                )

        return self

In [24]:
from scipy.special import expit  # Numerically stable sigmoid function


class FactorizationMachineClassifier1(BaseEstimator, ClassifierMixin):
    def __init__(
        self,
        rank: int = 16,
        n_epochs: int = 10,
        learning_rate: float = 0.05,
        l2_w: float = 1e-6,
        l2_V: float = 1e-6,
        init_stdev: float = 0.1,
        fit_intercept: bool = True,
        shuffle: bool = True,
        random_state: int = RANDOM_STATE,
        verbose: int = 1,
    ) -> None:
        self.rank = rank
        self.n_epochs = n_epochs
        self.learning_rate = learning_rate
        self.l2_w = l2_w
        self.l2_V = l2_V
        self.init_stdev = init_stdev
        self.fit_intercept = fit_intercept
        self.shuffle = shuffle
        self.random_state = random_state
        self.verbose = verbose

    def _init_params(self, n_features: int, rng: np.random.RandomState) -> None:
        self.w0_ = 0.0 if self.fit_intercept else 0.0
        self.w_ = np.zeros(n_features, dtype=np.float64)
        self.V_ = rng.normal(
            scale=self.init_stdev, size=(n_features, self.rank)
        ).astype(np.float64)

    def decision_function(self, X: sp.csr_matrix) -> np.ndarray:
        check_is_fitted(self, attributes=["w_", "V_"])

        # 1. Intercept term
        eta = self.w0_ if self.fit_intercept else 0.0

        # 2. Linear term: X @ w
        eta += X @ self.w_

        # 3. Pairwise interaction term: 0.5 * sum_k( (X @ V_k)^2 - (X^2 @ V_k^2) )
        # To avoid creating a dense matrix, this is calculated efficiently.
        term1 = (X @ self.V_) ** 2

        # Element-wise square of sparse matrix data
        X_sq = X.copy()
        X_sq.data **= 2

        term2 = X_sq @ (self.V_**2)

        eta += 0.5 * np.sum(term1 - term2, axis=1)

        return eta

    def predict_proba(self, X: sp.csr_matrix) -> np.ndarray:
        eta = self.decision_function(X)

        p1 = expit(eta)

        return np.vstack([1.0 - p1, p1]).T

    def fit(self, X: sp.csr_matrix, y: np.ndarray) -> Self:
        # X, y = validate_data(X, y)

        # Check that y is binary
        if len(np.unique(y)) != 2:
            raise ValueError("y must be binary {0,1}.")

        n_samples, n_features = X.shape
        rng = check_random_state(self.random_state)
        self._init_params(n_features, rng)

        order = np.arange(n_samples)
        lr = float(self.learning_rate)

        for epoch in range(self.n_epochs):
            if self.shuffle:
                rng.shuffle(order)

            total_loss = 0.0

            for r in order:
                start, end = X.indptr[r], X.indptr[r + 1]
                idx = X.indices[start:end]
                dat = X.data[start:end]

                # Skip empty rows
                if idx.size == 0:
                    continue

                # --- Forward Pass (for a single instance) ---
                eta = self.w0_

                # Linear term
                eta += np.dot(self.w_[idx], dat)

                # Interaction term (optimized calculation)
                V_slice = self.V_[idx]
                p_k = dat @ V_slice  # (rank,)
                interaction_term = 0.5 * (
                    np.sum(p_k**2) - np.sum((dat[:, None] * V_slice) ** 2)
                )
                eta += interaction_term

                # --- Gradient Calculation ---
                p = expit(eta)
                e = p - y[r]  # Gradient of log-loss w.r.t. eta

                # Accumulate log loss (with a small epsilon for stability)
                total_loss += -(
                    y[r] * np.log(p + 1e-15) + (1 - y[r]) * np.log(1 - p + 1e-15)
                )

                # --- Parameter Updates ---
                if self.fit_intercept:
                    self.w0_ -= lr * e

                # Update w (linear terms)
                grad_w = e * dat + self.l2_w * self.w_[idx]
                self.w_[idx] -= lr * grad_w

                # Update V (interaction terms)
                # IMPROVEMENT: More idiomatic and potentially faster in-place update.
                grad_V_slice = (
                    e * (dat[:, None] * (p_k - V_slice * dat[:, None]))
                    + self.l2_V * V_slice
                )
                self.V_[idx] -= lr * grad_V_slice

            if self.verbose:
                print(
                    f"[FM] Epoch {epoch + 1}/{self.n_epochs}, "
                    f"LogLoss: {total_loss / n_samples:.6f}"
                )

        return self

In [25]:
fm = FactorizationMachineClassifier1(
    rank=16,
    n_epochs=9,
    learning_rate=0.01,
    verbose=1,
)
fm_pipeline = Pipeline(
    steps=[
        ("features", featurizer),
        ("clf", fm),
    ]
)

fm_pipeline.fit(X_train, y_train)
y_pred_proba = fm_pipeline.predict_proba(X_test)[:, 1]
y_pred = (y_pred_proba >= 0.5).astype(int)

print("LogLoss (FM):", log_loss(y_test, y_pred_proba))
print("ROC-AUC (FM):", roc_auc_score(y_test, y_pred_proba))
print("PR-AUC  (FM):", average_precision_score(y_test, y_pred_proba))
print("Accuracy  (FM):", accuracy_score(y_test, y_pred))

KeyError: np.int64(61384)

#### Metrics / Plots

---

Use these helpers to evaluate (logloss, ROC-AUC, PR-AUC) and visualize learning curves when available.

In [11]:
# # pip install plotly
# import numpy as np
# import plotly.graph_objects as go
# from sklearn.metrics import log_loss, roc_auc_score, average_precision_score


# def evaluate_scores(y_true, y_pred_proba, label: str = "model", show: bool = True):
#     """
#     Compute LogLoss / ROC-AUC / PR-AUC and display them as a Plotly table.
#     Accepts either a 1D array of positive-class probabilities or a 2D array with [:,1].
#     """
#     y_pred = (
#         y_pred_proba[:, 1] if getattr(y_pred_proba, "ndim", 1) == 2 else y_pred_proba
#     )

#     metrics = {
#         "Model": [label],
#         "LogLoss": [log_loss(y_true, y_pred)],
#         "ROC-AUC": [roc_auc_score(y_true, y_pred)],
#         "PR-AUC": [average_precision_score(y_true, y_pred)],
#     }

#     fig = go.Figure(
#         data=[
#             go.Table(
#                 header=dict(
#                     values=list(metrics.keys()), fill_color="#f5f5f5", align="left"
#                 ),
#                 cells=dict(values=list(metrics.values()), align="left"),
#             )
#         ]
#     )
#     fig.update_layout(
#         title=f"Evaluation Metrics — {label}", margin=dict(l=10, r=10, t=40, b=10)
#     )

#     if show:
#         print(f"[{label}] LogLoss: {metrics['LogLoss'][0]:.6f}")
#         print(f"[{label}] ROC-AUC: {metrics['ROC-AUC'][0]:.6f}")
#         print(f"[{label}] PR-AUC : {metrics['PR-AUC'][0]:.6f}")
#         fig.show()

#     return metrics, fig


# def plot_learning_curve(xs, ys):
#     """
#     Interactive learning curve with best-epoch marker.
#     xs: list/array of epochs
#     ys: list/array of logloss values
#     """
#     xs = np.asarray(xs)
#     ys = np.asarray(ys)
#     best_idx = int(np.argmin(ys))

#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=xs, y=ys, mode="lines+markers", name="logloss"))
#     fig.add_trace(
#         go.Scatter(
#             x=[xs[best_idx]],
#             y=[ys[best_idx]],
#             mode="markers+text",
#             text=[f"best={ys[best_idx]:.5f} @ {xs[best_idx]}"],
#             textposition="top center",
#             marker=dict(size=10, symbol="star"),
#         )
#     )
#     fig.update_layout(
#         title="LogLoss vs Epoch",
#         xaxis_title="epoch",
#         yaxis_title="logloss",
#         template="plotly_white",
#         margin=dict(l=10, r=10, t=40, b=10),
#     )

#     fig.show()


# # metrics, table_fig = evaluate_scores(y_train, y_pred_proba)
# lc_fig = plot_learning_curve(range(1, len(10) + 1), 10)


## 9. Practical tips (Avazu-scale)
- **Split:** by day (`hour`), hold-out last day for validation.
- **Hashing:** start with `2**22` or `2**24` features. Increase if collisions are harmful.
- **FM rank (`k`)**: 8–32 are common for CTR. Tune with validation.
- **Regularization:** small `l2_reg_w`, `l2_reg_V` (e.g., `1e-6..1e-3`) for FM; `C`/penalty for LR.
- **Learning rate:** `fastFM` `step_size` ~ `1e-2..1e-1`; `xlearn` `lr` ~ `0.05..0.3`.
- **Batching:** stream CSV in chunks; keep validation separate.
- **FFM:** often beats FM on CTR when fields are well-defined (user/app/device/site…).  
- **Calibration:** if you need perfectly calibrated CTRs, consider Platt/Isotonic on the validation set.
